In [ ]:
from __future__ import annotations

import json
import os
from datetime import datetime
from typing import Any, Dict, Optional, Tuple

import requests
from dotenv import load_dotenv

load_dotenv()

In [ ]:
LAW_SEARCH_URL = "http://www.law.go.kr/DRF/lawSearch.do"
LAW_SERVICE_URL = "http://www.law.go.kr/DRF/lawService.do"

In [3]:
def _require_oc() -> str:
    oc = os.getenv("LAW_OC", "").strip()
    if not oc:
        raise RuntimeError(
            "환경변수 LAW_OC 가 비어있습니다. 예) Windows: set LAW_OC=g4c  /  macOS/Linux: export LAW_OC=g4c"
        )
    return oc

In [2]:
def _get_json(url: str, params: Dict[str, Any], timeout: int = 30) -> Dict[str, Any]:
    r = requests.get(url, params=params, timeout=timeout)
    r.raise_for_status()

    # DRF는 content-type이 애매하게 오는 경우가 있어서 강제로 json 파싱 시도
    try:
        return r.json()
    except Exception as e:
        # 혹시 에러/HTML 응답이면 원문 일부를 보여주기 위해
        text_head = (r.text or "")[:500]
        raise RuntimeError(f"JSON 파싱 실패. 응답 앞부분:\n{text_head}") from e

In [4]:
def find_latest_eflaw(oc: str, law_name: str) -> Tuple[Optional[str], Optional[str], Optional[str], Dict[str, Any]]:
    """
    주어진 법령명으로 eflaw 목록 검색 후 최신 시행일(현행) 1건을 뽑는다.

    반환:
    - law_id: (법령ID) -> lawService의 ID로도 쓰일 수 있음
    - mst: (법령일련번호/lsi_seq 추정) -> lawService의 MST로 사용
    - ef_yd: (시행일자) -> lawService의 efYd로 사용
    - raw_item: 선택된 법령 row 원본
    """
    params = {
        "OC": oc,
        "target": "eflaw",
        "type": "JSON",
        "query": law_name,
        "nw": 3,          # 3: 현행
        "sort": "efdes",  # 시행일자 내림차순
        "display": 1,
        "page": 1,
    }

    data = _get_json(LAW_SEARCH_URL, params=params)

    # 응답 구조가 케이스별로 달라질 수 있어 안전하게 law 리스트를 찾는다.
    # 보통 data["LawSearch"]["law"] 형태이거나, data["law"] 형태로 오는 경우가 있음.
    law_list = None
    if isinstance(data, dict):
        if "LawSearch" in data and isinstance(data["LawSearch"], dict) and "law" in data["LawSearch"]:
            law_list = data["LawSearch"]["law"]
        elif "law" in data:
            law_list = data["law"]

    if not law_list:
        # totalCnt가 0이거나 파싱 실패일 수 있음
        return None, None, None, {"_raw": data}

    # law_list가 dict 1개로 올 수도, list로 올 수도 있음
    if isinstance(law_list, dict):
        item = law_list
    elif isinstance(law_list, list):
        item = law_list[0]
    else:
        return None, None, None, {"_raw": data}

    # 필드명은 가이드에 한글 키로도 나오고, 영문/축약 형태로도 나올 수 있음.
    # 가능한 키들을 넓게 커버.
    law_id = (
        str(item.get("법령ID") or item.get("lawId") or item.get("ID") or item.get("id") or "")
        .strip()
        or None
    )

    mst = (
        str(item.get("법령일련번호") or item.get("mst") or item.get("MST") or item.get("lsiSeq") or "")
        .strip()
        or None
    )

    ef_yd = (
        str(item.get("시행일자") or item.get("efYd") or item.get("EFYD") or "")
        .strip()
        or None
    )

    return law_id, mst, ef_yd, item

In [5]:
def fetch_eflaw_body_json(
    oc: str,
    *,
    law_id: Optional[str],
    mst: Optional[str],
    ef_yd: Optional[str],
    original_text: bool = True,
    jo: Optional[str] = None,
) -> Dict[str, Any]:
    """
    eflaw 본문 조회(JSON)

    - ID로 조회하면 efYd는 무시된다고 가이드에 명시되어 있음.
    - MST로 조회할 경우 efYd가 필수.

    original_text:
      - True  -> chrClsCd=010201 (원문)
      - False -> chrClsCd=010202 (한글)
    """
    params: Dict[str, Any] = {
        "OC": oc,
        "target": "eflaw",
        "type": "JSON",
        "chrClsCd": "010201" if original_text else "010202",
    }
    if jo:
        params["JO"] = jo

    # 1) 가능한 경우 ID로 먼저 시도 (간단)
    if law_id:
        params_id = dict(params)
        params_id["ID"] = law_id
        try:
            return _get_json(LAW_SERVICE_URL, params=params_id)
        except Exception:
            # ID 방식이 실패하면 MST 방식으로 fallback
            pass

    # 2) MST+efYd 방식
    if not mst or not ef_yd:
        raise RuntimeError(f"MST/efYd 정보가 부족합니다. mst={mst}, ef_yd={ef_yd}")

    params_mst = dict(params)
    params_mst["MST"] = mst
    params_mst["efYd"] = ef_yd
    return _get_json(LAW_SERVICE_URL, params=params_mst)

In [6]:
def main():
    oc = _require_oc()

    law_name = "주택임대차보호법"

    law_id, mst, ef_yd, picked = find_latest_eflaw(oc, law_name)
    if not (law_id or (mst and ef_yd)):
        raise RuntimeError(f"법령을 찾지 못했습니다. 검색결과: {json.dumps(picked, ensure_ascii=False)[:500]}")

    body = fetch_eflaw_body_json(
        oc,
        law_id=law_id,
        mst=mst,
        ef_yd=ef_yd,
        original_text=True,  # 원문(한자 포함) 원하면 True, 한글만이면 False
        jo=None,             # 특정 조만 원하면 예: "000200"
    )

    # 파일명: 시행일자(있으면) + 저장시각
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    tag = ef_yd or "unknown_efyd"
    out_path = f"주택임대차보호법_eflaw_{tag}_{now}.json"

    payload = {
        "meta": {
            "law_name": law_name,
            "picked_law_id": law_id,
            "picked_mst": mst,
            "picked_efYd": ef_yd,
            "picked_row": picked,
            "saved_at": now,
        },
        "body": body,
    }

    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)

    print(f"[OK] saved: {out_path}")


if __name__ == "__main__":
    main()

RuntimeError: 환경변수 LAW_OC 가 비어있습니다. 예) Windows: set LAW_OC=g4c  /  macOS/Linux: export LAW_OC=g4c